In [1]:
from google.colab import drive
import sys
import os
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/MyDrive/My_project_v2')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
pip install pretty_midi

In [3]:
pip install transformers

In [4]:
!cp drive/MyDrive/My_project_v2/vq_vae.py

cp: missing destination file operand after 'drive/MyDrive/My_project_v2/vq_vae.py'
Try 'cp --help' for more information.


In [5]:
import os
from vq_vae import vq_vae
from seq2seq import Seq2SeqModule
from datamodule import Mydatamodule,Mydataset,SeqCollator


import time
import glob
from torch import optim
import torch
from torch import nn
from torch.utils.data import DataLoader
import transformers
import math

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


def train_mymodel(epoch_num, model, train_dataloader, val_dataloader, optimizer, scheduler):

    print('start_training')
    lr_warmup_steps = 200
    max_lr = 0.0001
    trained_steps = 0

    for epoch in range(epoch_num):
        scheduler.step()
        print('epoch:',epoch)
        model.train()
        for batch_idx, batch_samples in enumerate(train_dataloader):

            trained_steps += 1
            #batch_samples.to(device)

            metrics = model.get_loss(batch_samples,device)

            log_metrics = {key: metrics[key].detach() for key in
                           ['loss', 'rec_loss', 'aa','diff', 'avg_usage', 'usage', 'entropy'] if key in metrics}

            optimizer.zero_grad()
            metrics['loss'].backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
              # aa = log_metrics['aa']
              # o2 = aa.argmax(dim=2)
              # label = batch_samples['labels']
              # print('pred------------')
              # print(o2[0])
              # print('label-----------')
              # print(label[0])
              print('train_loss:',log_metrics['loss'].item())
            if batch_idx % 800 == 0:
              print('example:')
              aa = log_metrics['aa']
              o2 = aa.argmax(dim=2)
              label = batch_samples['labels']
              print('pred------------')
              print(o2[0])
              print('label-----------')
              print(label[0])

            if batch_idx % 200 == 0:
              print('save')
              torch.save(model.state_dict(), '/content/gdrive/MyDrive/My_project_v2/vqvae_weights.pt')
        model.eval()
        e_n = 0
        print('evaluating---')
        for batch_idx, batch_samples in enumerate(val_dataloader):
            e_n += 1
            if e_n == 3:
              break
            metrics = model.get_loss(batch_samples,device)
            log_metrics = {key: metrics[key].detach() for key in
                           ['loss', 'rec_loss', 'aa','diff', 'avg_usage', 'usage', 'entropy'] if key in metrics}
            print('eval_loss:',log_metrics['loss'].item())


        

In [7]:
def main():

    MAX_N_FILES = -1
    #ROOT_DIR = '/content/gdrive/MyDrive/My_project_v2/POP909/'
    ROOT_DIR = '/content/gdrive/MyDrive/my_project/data/'

    midi_files = glob.glob(os.path.join(ROOT_DIR, '**/*.midi'),recursive=False)

    if MAX_N_FILES > 0:
        midi_files = midi_files[:MAX_N_FILES]
    if len(midi_files) == 0:
        print(f"WARNING: No MIDI files were found at '{ROOT_DIR}'. Did you download the dataset to the right location?")
        return 0

    vae_module = vq_vae()
    #vae_module.load_state_dict(torch.load('/content/gdrive/MyDrive/My_project_v2/vqvae_weights.pt'))
    vae_module.to(device)

    mydatamodule = Mydatamodule(midi_files,256,vae_module,max_bars=1024,max_bars_per_context=1)
    #mydatamodule = Mydatamodule(midi_files,256,vae_module,max_bars=1024)
    mydatamodule.setup()

    train_dataloader = mydatamodule.train_dataloader()
    val_dataloader = mydatamodule.val_dataloader()
    test_dataloader = mydatamodule.test_dataloader()

    n_params = sum(p.numel() for p in vae_module.parameters() if p.requires_grad)
    print('[info] model # params:', n_params)

    opt_params = filter(lambda p: p.requires_grad, vae_module.parameters())
    #optimizer = torch.optim.Adam(opt_params, lr=1)

    optimizer = torch.optim.AdamW(vae_module.parameters(), lr=1, weight_decay=0.01)
    
    #lr_func = lambda step: 0.0001
    #scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_func)


    #scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 100000, eta_min=2e-06)

    lr_func = lambda step: min(1e-4, 1e-4 / math.sqrt(max(step, 1) / 10))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_func)
    
    train_mymodel(50,vae_module,train_dataloader,val_dataloader,optimizer,scheduler)
    # k=1
    # for i in train_dataloader:
    #     k += 1
    #     print(k)
    #     #metrics = model.get_loss(i, device)
    #     print(i['input_ids'].shape)
    #     if k == 4:
    #         break


In [8]:
#main()

In [9]:
def train_mymodel_seq2seq(epoch_num, model, train_dataloader, val_dataloader, optimizer, scheduler):

    print('start_training')
    lr_warmup_steps = 200
    max_lr = 0.0001
    trained_steps = 0

    for epoch in range(epoch_num):

        scheduler.step()
        print('epoch:',epoch)

        model.train()
        for batch_idx, batch_samples in enumerate(train_dataloader):

            trained_steps += 1
            #batch_samples.to(device)

            loss = model.get_loss(batch_samples,device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % 50 == 0:
              # aa = log_metrics['aa']
              # o2 = aa.argmax(dim=2)
              # label = batch_samples['labels']
              # print('pred------------')
              # print(o2[0])
              # print('label-----------')
              # print(label[0])
              print(loss.item())
              torch.save(model.state_dict(), '/content/gdrive/MyDrive/My_project_v2/seq2seq_weights.pt')


        model.eval()
        e_n = 0
        print('evaluating---')
        for batch_idx, batch_samples in enumerate(val_dataloader):
            e_n += 1
            if e_n == 3:
              break
            loss = model.get_loss(batch_samples,device)
            print('eval_loss:',loss.item())


        

In [10]:
def main_seq2seq():

    MAX_N_FILES = -1
    # ROOT_DIR = '/content/gdrive/MyDrive/My_project_v2/POP909/'
    # midi_files = glob.glob(os.path.join(ROOT_DIR, '**/*.mid'),recursive=False)

    ROOT_DIR = '/content/gdrive/MyDrive/my_project/data/'
    midi_files = glob.glob(os.path.join(ROOT_DIR, '**/*.midi'),recursive=True)

    if MAX_N_FILES > 0:
        midi_files = midi_files[:MAX_N_FILES]
    if len(midi_files) == 0:
        print(f"WARNING: No MIDI files were found at '{ROOT_DIR}'. Did you download the dataset to the right location?")
        return 0

    vae_module = vq_vae()
    vae_module.load_state_dict(torch.load('/content/gdrive/MyDrive/My_project_v2/vqvae_weights.pt',map_location=torch.device('cpu')))

    #vae_module.to(device)

    for param in vae_module.parameters():
      param.requires_grad = False

    #print(midi_files)


    seq2seq = Seq2SeqModule().to(device)

    #mydatamodule = Mydatamodule(midi_files,256,vae_module,max_bars=1024,max_bars_per_context=1)
    mydatamodule = Mydatamodule(midi_files,256,vae_module,max_bars=1024,description_flavor='latent')
    mydatamodule.setup()

    train_dataloader = mydatamodule.train_dataloader()
    val_dataloader = mydatamodule.val_dataloader()
    test_dataloader = mydatamodule.test_dataloader()

    n_params = sum(p.numel() for p in seq2seq.parameters() if p.requires_grad)
    print('[info] model # params:', n_params)

    opt_params = filter(lambda p: p.requires_grad, seq2seq.parameters())
    #optimizer = torch.optim.Adam(opt_params, lr=1)

    #optimizer = transformers.AdamW(seq2seq.parameters(), lr=1, weight_decay=0.01)
    
    #scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 100000, eta_min=2e-06)

    #lr_func = lambda step: min(1e-4, 1e-4 / math.sqrt(max(step, 1) / 5000))
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_func)

    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, 1e-4)
  
    optimizer = torch.optim.AdamW(seq2seq.parameters(), lr=1, weight_decay=0.01)
  

    lr_func = lambda step: min(1e-4, 1e-4 / math.sqrt(max(step, 1) / 10))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_func)

    train_mymodel_seq2seq(50,seq2seq,train_dataloader,val_dataloader,optimizer,scheduler)

    k=1
    for i in train_dataloader:
        k += 1
        print(k)
        #metrics = model.get_loss(i, device)
        print(i['latents'].shape)

        if k == 10:
            break


In [11]:
#main_seq2seq()

In [12]:
#generate

In [13]:
from util.input_representation import remi2midi

In [14]:
MODEL = os.getenv('MODEL', '')

ROOT_DIR = os.getenv('ROOT_DIR', './lmd_full')
OUTPUT_DIR = os.getenv('OUTPUT_DIR', './samples')
MAX_N_FILES = int(float(os.getenv('MAX_N_FILES', -1)))
MAX_ITER = int(os.getenv('MAX_ITER', 16_000))
MAX_BARS = int(os.getenv('MAX_BARS', 32))

MAKE_MEDLEYS = os.getenv('MAKE_MEDLEYS', 'False') == 'True'
N_MEDLEY_PIECES = int(os.getenv('N_MEDLEY_PIECES', 2))
N_MEDLEY_BARS = int(os.getenv('N_MEDLEY_BARS', 16))
  
CHECKPOINT = os.getenv('CHECKPOINT', None)
VAE_CHECKPOINT = os.getenv('VAE_CHECKPOINT', None)
BATCH_SIZE = int(os.getenv('BATCH_SIZE', 1))
VERBOSE = int(os.getenv('VERBOSE', 2))

In [15]:
def reconstruct_sample(model, batch, 
  initial_context=1, 
  output_dir=None, 
  max_iter=-1, 
  max_bars=-1,
  verbose=0,
):
  batch_size, seq_len = batch['input_ids'].shape[:2]

  batch_ = { key: batch[key][:, :initial_context] for key in ['input_ids', 'bar_ids', 'position_ids'] }
  if model.description_flavor in ['description', 'both']:
    batch_['description'] = batch['description']
    batch_['desc_bar_ids'] = batch['desc_bar_ids']
  if model.description_flavor in ['latent', 'both']:
    batch_['latents'] = batch['latents']

  max_len = seq_len + 512
  max_len = 800
  if max_iter > 0:
    max_len = min(max_len, initial_context + max_iter)
  if verbose:
    print(f"Generating sequence ({initial_context} initial / {max_len} max length / {max_bars} max bars / {batch_size} batch size)")
  sample = model.sample(batch_, max_length=max_len, max_bars=max_bars, verbose=verbose//2)

  xs = batch['input_ids'].detach().cpu()
  xs_hat = sample['sequences'].detach().cpu()
  events = [model.vocab.decode(x) for x in xs]
  events_hat = [model.vocab.decode(x) for x in xs_hat]

  pms, pms_hat = [], []
  n_fatal = 0
  for rec, rec_hat in zip(events, events_hat):
    try:
      pm = remi2midi(rec)
      pms.append(pm)
    except Exception as err:
      print("ERROR: Could not convert events to midi:", err)
    try:
      pm_hat = remi2midi(rec_hat)
      pms_hat.append(pm_hat)
    except Exception as err:
      print("ERROR: Could not convert events to midi:", err)
      n_fatal += 1

  if output_dir:
    os.makedirs(os.path.join(output_dir, 'gt'), exist_ok=True)
    for pm, pm_hat, file in zip(pms, pms_hat, batch['files']):
      if verbose:
        print(f"Saving to {output_dir}/{file}")
      pm.write(os.path.join(output_dir, 'gt', file))
      pm_hat.write(os.path.join(output_dir, file))

  return events



In [16]:
from torch.nn.utils.rnn import pad_sequence
def combine_batches(batches, bars_per_sequence=8, description_flavor='none', device=None):
  if device is None:
    device = batches[0]['input_ids'].device

  batch_size = batches[0]['input_ids'].size(0)

  zero = torch.zeros(1, device=device, dtype=torch.int)

  contexts = []
  batch_ = {}

  for i in range(batch_size):
    curr_bar = 0
    ctx = {
      'input_ids': [],
      'bar_ids': [],
      'position_ids': [],
      'slices': [],
      'description': [],
      'desc_bar_ids': [],
      'desc_slices': [],
      'latents': [],
      'latent_slices': [],
      'files': [],
    }

    for batch in batches:
      if i >= batch['input_ids'].size(0):
        continue

      curr = curr_bar

      bar_ids = batch['bar_ids'][i]
      starts = (bar_ids >= curr).nonzero()
      ends = (bar_ids >= max(1, curr) + bars_per_sequence).nonzero()
      if starts.size(0) == 0:
        continue
      start = starts[0, 0]

      if ends.size(0) == 0:
        end = bar_ids.size(0)
        curr_bar = bar_ids[-1] + 1
      else:
        end = ends[0, 0]
        curr_bar = bar_ids[end]

      if description_flavor in ['description', 'both']:
        desc_bar_ids = batch['desc_bar_ids'][i]
        desc_start = (desc_bar_ids >= curr).nonzero()[0, 0]
        desc_ends = (desc_bar_ids >= max(1, curr) + bars_per_sequence).nonzero()

        if desc_ends.size(0) == 0:
          desc_end = desc_bar_ids.size(0)
        else:
          desc_end = desc_ends[0, 0]

      if description_flavor in ['latent', 'both']:
        latent_start = curr
        latent_end = max(1, curr) + bars_per_sequence


      ctx['input_ids'].append(batch['input_ids'][i, start:end])
      ctx['bar_ids'].append(batch['bar_ids'][i, start:end])
      ctx['position_ids'].append(batch['position_ids'][i, start:end])
      ctx['slices'].append((start, end))
      if description_flavor in ['description', 'both']:
        ctx['description'].append(batch['description'][i, desc_start:desc_end])
        ctx['desc_bar_ids'].append(batch['desc_bar_ids'][i, desc_start:desc_end])
        ctx['desc_slices'].append((desc_start, desc_end))
      if description_flavor in ['latent', 'both']:
        ctx['latents'].append(batch['latents'][i, latent_start:latent_end])
        ctx['latent_slices'].append((latent_start, latent_end))
      ctx['files'].append(batch['files'][i])

    if len(ctx['files']) <= 1:
      continue
  
    keys = ['input_ids', 'bar_ids', 'position_ids', 'description', 'desc_bar_ids', 'latents']
    for key in keys:
      if key in ctx and len(ctx[key]) > 0:
        ctx[key] = torch.cat(ctx[key])
    ctx['labels'] = torch.cat([ctx['input_ids'][1:], zero])
    ctx['files'] = '__'.join(ctx['files']).replace('.mid', '') + '.mid'

    contexts.append(ctx)

  batch_['files'] = [ctx['files'] for ctx in contexts]

  for key in ['input_ids', 'bar_ids', 'position_ids', 'description', 'desc_bar_ids', 'latents', 'labels']:
    xs = [ctx[key] for ctx in contexts if isinstance(ctx[key], torch.Tensor)]
    if len(xs) > 0:
      xs = pad_sequence(xs, batch_first=True, padding_value=0)
      if not key in ['latents']:
        xs = xs.long()
      batch_[key] = xs

  return batch_


def medley_iterator(dl, n_pieces=2, n_bars=8, description_flavor='none'):
  dl_iter = iter(dl)
  try:
    while True:
      batches = [next(dl_iter) for _ in range(n_pieces)]
      batch = combine_batches(batches, 
        bars_per_sequence=n_bars, 
        description_flavor=description_flavor
      )
      yield batch
  except StopIteration:
    return

In [17]:
def generate():
  if MAKE_MEDLEYS:
    max_bars = N_MEDLEY_PIECES * N_MEDLEY_BARS
  else:
    max_bars = MAX_BARS

  if OUTPUT_DIR:
    params = []
    if MAKE_MEDLEYS:
      params.append(f"n_pieces={N_MEDLEY_PIECES}")
      params.append(f"n_bars={N_MEDLEY_BARS}")
    if MAX_ITER > 0:
      params.append(f"max_iter={MAX_ITER}")
    if MAX_BARS > 0:
      params.append(f"max_bars={MAX_BARS}")
    output_dir = os.path.join(OUTPUT_DIR, MODEL, ','.join(params))
    output_dir = '/content/gdrive/MyDrive/my_project/sample/'
  else:
    raise ValueError("OUTPUT_DIR must be specified.")

  print(f"Saving generated files to: {output_dir}")

  vae_module = vq_vae()
  vae_module.load_state_dict(torch.load('/content/gdrive/MyDrive/My_project_v2/vqvae_weights.pt',map_location=torch.device('cpu')))
  for param in vae_module.parameters():
    param.requires_grad = False

  model = Seq2SeqModule(device='cpu')
  model.load_state_dict(torch.load('/content/gdrive/MyDrive/My_project_v2/seq2seq_weights.pt',map_location=torch.device('cpu')))

  for param in model.parameters():
    param.requires_grad = False


  ROOT_DIR = '/content/gdrive/MyDrive/my_project/data/'
  midi_files = glob.glob(os.path.join(ROOT_DIR, '**/*.midi'),recursive=True)
  midi_files = midi_files[:20]

  dataset = Mydataset(
    midi_files,
    max_len=-1,
    description_flavor='latent',
    description_options=None,
    max_bars=512,
    vae_module=vae_module
  )


  start_time = time.time()
  coll = SeqCollator(context_size=-1)
  dl = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=coll)

  if MAKE_MEDLEYS:
    dl = medley_iterator(dl, 
      n_pieces=N_MEDLEY_BARS, 
      n_bars=N_MEDLEY_BARS, 
      description_flavor=model.description_flavor
    )
  
  with torch.no_grad():
    for batch in dl:
      reconstruct_sample(model, batch, 
        output_dir=output_dir, 
        max_iter=MAX_ITER, 
        max_bars=max_bars,
        verbose=VERBOSE,
      )


In [18]:
generate()

流式输出内容被截断，只能显示最后 5000 行内容。
395/800 ['Position_32']
396/800 ['Instrument_Acoustic Grand Piano']
397/800 ['Pitch_84']
398/800 ['Velocity_17']
399/800 ['Duration_15']
400/800 ['Position_41']
401/800 ['Instrument_Acoustic Grand Piano']
402/800 ['Pitch_75']
403/800 ['Velocity_16']
404/800 ['Duration_14']
405/800 ['Position_41']
406/800 ['Instrument_Acoustic Grand Piano']
407/800 ['Pitch_80']
408/800 ['Velocity_16']
409/800 ['Duration_14']
410/800 ['Position_42']
411/800 ['Instrument_Acoustic Grand Piano']
412/800 ['Pitch_68']
413/800 ['Velocity_11']
414/800 ['Duration_14']
415/800 ['Position_47']
416/800 ['Instrument_Acoustic Grand Piano']
417/800 ['Pitch_70']
418/800 ['Velocity_16']
419/800 ['Duration_14']
420/800 ['Position_47']
421/800 ['Instrument_Acoustic Grand Piano']
422/800 ['Pitch_79']
423/800 ['Velocity_17']
424/800 ['Duration_8']
425/800 ['Bar_9']
426/800 ['Time Signature_4/4']
427/800 ['Position_0']
428/800 ['Chord_F:min7']
429/800 ['Position_0']
430/800 ['Tempo_4']
431/800 ['Po

KeyboardInterrupt: ignored

In [ ]:
device